In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/data-analyst-jobs/DataAnalyst.csv",index_col = 0)
df[:2]

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
# Find lower and upper bounds of salaries
df["Salary Estimate"] = df["Salary Estimate"].str.replace("\(Glassdoor est.\)", "")
df["Salary Estimate"] = df["Salary Estimate"].str.replace("K", "")
df[['Salary Lower bound','Salary Upper bound']] = df["Salary Estimate"].str.split("-",expand=True)

df["Salary Lower bound"] = df["Salary Lower bound"].str.replace("$", "")
df["Salary Upper bound"] = df["Salary Upper bound"].str.replace("$", "")


# df["Salary Lower bound"] = df["Salary Lower bound"].str.replace(" ", "0")
# df["Salary Upper bound"] = df["Salary Upper bound"].str.replace(" ", "0")

df["Salary Lower bound"] = pd.to_numeric(df["Salary Lower bound"])
df["Salary Upper bound"] = pd.to_numeric(df["Salary Upper bound"])

df[:2]

In [ ]:
df[["City", "State", "None"]] = df["Location"] = pd.DataFrame(df.Location.str.split(",", expand = True))
df[:2]

In [ ]:
# Rectify. CO should be in "State column"

# Check affected rows
arapahoe = df["State"].str.contains("Arapahoe")
df[arapahoe]

df["State"] = df["State"].str.replace("Arapahoe","CO")

In [ ]:
# Jobs per location
location = df.groupby("Location").count()
location[["Job Title"]].sort_values(by = "Job Title", ascending = False)


In [ ]:
# Location with 10 or less jobs
less_than_10_jobs = location.loc[location["Job Title"] <= 10]
less_than_10_jobs = less_than_10_jobs[["Job Title"]]
less_than_10_jobs.rename(columns = {"Job Title":"Jobs"}, inplace = True)
less_than_10_jobs[:5]

In [ ]:
# Location with more than 10 jobs
more_than_10_jobs = location.loc[location["Job Title"] > 10]
more_than_10_jobs = more_than_10_jobs[["Job Title"]]
more_than_10_jobs.rename(columns = {"Job Title":"Jobs"}, inplace = True)
more_than_10_jobs

In [ ]:
jobs = more_than_10_jobs["Jobs"].tolist()
locations = more_than_10_jobs.index.tolist()
locations_df = pd.DataFrame()
locations_df["location"] = locations
locations_df["jobs"] = jobs
locations_df

In [ ]:
# New row to add other the other 211 locations
new_row = {'location': "Other", "jobs":int(less_than_10_jobs.count())}
#append row to the dataframe
locations_df = locations_df.append(new_row, ignore_index=True)
locations_df[-5:]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style

style.use("bmh")

In [ ]:
# Jobs per location
locations_df.plot(figsize = (18, 8), kind = "bar", x = "location", y = "jobs")
plt.title("Data analyst jobs per location")
plt.ylabel("Number of jobs")

In [ ]:
# Jobs per state
state = df.groupby("State").count()
state = state["Job Title"]
state.plot(figsize = (18,8), kind = "bar")
plt.title("Data analyst jobs per state")
plt.ylabel("Number of jobs")

In [ ]:
# Jobs per sector
sector = df.groupby("Sector").count()
sector = sector["Job Title"]
sector.plot(figsize = (18,8), kind = "bar")
plt.title("Data analyst jobs per sector")
plt.ylabel("Number of jobs")

In [ ]:
# Jobs per industry
industry = df.groupby("Industry").count()
industry = industry["Job Title"]
industry.plot(figsize = (18,8), kind = "bar")
plt.title("Data analyst jobs per industry")
plt.ylabel("Number of jobs")

In [ ]:
# Jobs per Type of ownership
ownership = df.groupby("Type of ownership").count()
ownership = ownership["Job Title"]
ownership.plot(figsize = (18,8), kind = "bar")
plt.title("Data analyst jobs per Type of Ownership")
plt.ylabel("Number of jobs")

In [ ]:
# Jobs per Easy Apply
easy_apply = df.groupby("Easy Apply").count()
easy_apply = easy_apply["Job Title"]
easy_apply.plot(figsize = (18,8), kind = "bar")
plt.title("Data analyst jobs per Easy Apply")
plt.ylabel("Number of jobs")

In [ ]:
# Jobs per Rating
rating = df.groupby("Rating").count()
rating = rating["Job Title"]
rating.plot(figsize = (18,8), kind = "bar")
plt.title("Data analyst jobs per Rating")
plt.ylabel("Number of jobs")


In [ ]:
# Jobs by companies with 5 star rating
five_star_rated_companies = df.loc[df["Rating"] == 5]
five_star_rated_companies.shape

In [ ]:
five_star_rated_companies[:2]

In [ ]:
# Top ten jobs with the highest upper bound salary
top_ten_upper_bound = five_star_rated_companies.sort_values(by = "Salary Upper bound", ascending = False)[:10]
top_ten_upper_bound[["Job Title", "Salary Estimate", "Company Name", "Location"]]

In [ ]:
# Top ten jobs with the highest lower bound salary
top_ten_lower_bound = five_star_rated_companies.sort_values(by = "Salary Lower bound", ascending = False)[:10]
top_ten_lower_bound[["Job Title", "Salary Estimate", "Company Name", "Location"]]

In [ ]:
# Highest paid junior jobs for five star companies - Upper bound
junior_mask = five_star_rated_companies["Job Title"].str.contains("Junior")
best_junior_jobs_ub = five_star_rated_companies[junior_mask].sort_values(by = "Salary Upper bound", ascending = False)[:10]
best_junior_jobs_ub[["Job Title", "Salary Estimate", "Company Name", "Location"]]

In [ ]:
# Highest paid junior jobs for five star companies - Lower bound
junior_mask = five_star_rated_companies["Job Title"].str.contains("Junior")
best_junior_jobs_lb = five_star_rated_companies[junior_mask].sort_values(by = "Salary Lower bound", ascending = False)[:10]
best_junior_jobs_lb[["Job Title", "Salary Estimate", "Company Name", "Location"]]

In [ ]:
# Highest median paying states
best_paying_jobs_state_ub = df.groupby("State").median().sort_values(by = "Salary Upper bound", 
                                                                  ascending = False)
best_paying_jobs_state_ub[["Salary Upper bound", "Salary Lower bound"]].plot(figsize = (18,8), kind = "bar")
plt.title("Top paying states for Data analyst jobs")
plt.ylabel("Median salary : Thousands")


In [ ]:
# Rating made per state
ratings_made_per_state = df.groupby("State").count()
ratings_made_per_state[["Rating"]].plot(figsize = (18,8), kind = "bar")
plt.title("Number of ratings on jobs per state")
plt.ylabel("Number of ratings")

In [ ]:
# Best job ratings per state
rating_per_state = df.groupby("State").median()
rating_per_state["Rating"].plot(figsize = (18,8), kind = "bar")
plt.title("Median job rating per state")
plt.ylabel("Rating")


In [ ]:
# Lets drill into "Georgia"
ga = df["State"].str.contains("GA")
df[ga]